# Week 13: Example Sentences Classification
In the last two assignment, we use deep learning method to do classification task. We can get good result by deep learning, but we can hardly explain the classification result(i.e., we don't know why the phrase is classified as "good" phrase). Hence, in this assignment, we want you do classification again, but do it by traditional machine learning method. In this way, you can know *why* more. We want you implement this on example sentences(e.g., "This song has a brilliant piano intro." for word "brilliant").

[Here](https://drive.google.com/drive/folders/1ij20ecLlI1Zh5CdMAa91SXQzmpKfdBdJ?usp=sharing) are two files needed for this task: **train.jsonl** & **test.jsonl**. In these files, each line records one example sentence with its label. There're two types of labels: ***positive*** means it's *good* example sentence;  ***negative*** means it's *bad* example sentence.

## Read Data
We use dataframe to keep data in this assignment.

In [1]:
import json
import pandas as pd

In [2]:
def read_data_to_df(path):
    labels = []
    sentences = []
    with open(path, 'r') as f:
        for line in f.readlines():
            line = json.loads(line)
            sentences.append(line['text'])
            labels.append(line['label'])
    return pd.DataFrame({'sent':sentences,'label':labels})  

In [3]:
train = read_data_to_df('data/train.jsonl')
print(train.head())
test = read_data_to_df('data/test.jsonl')
print(test.head())

                                                sent     label
0         My children threw a birthday party for me.  positive
1  Marketing on social networking sites is a fast...  positive
2                You pillock, look what you've done!  positive
3      He scored three touchdowns early in the game.  positive
4          His abrupt manner makes me uncomfortable.  positive
                                                sent     label
0  I've just spotted Mark - he's over there, near...  positive
1  After repricing, the bonds yield a much higher...  positive
2             I admire her staunchness and fidelity.  positive
3  The party's leader is in favour of the treaty ...  positive
4  About 20 companies are working on treatments f...  positive


## Extract Features
Traditional machine learning need data scientists to observe data and find out some useful information. 

Here is an example:

In [4]:
import re
# for positive
spec = 0
count=0
for i in train[train['label'] == "positive"]["sent"]:
    if len(re.sub('[\w\s]','',i)) >3:
        spec+=1
    count+=1
print(spec/count)

0.058014915590512126


In [5]:
spec = 0
count=0
for i in train[train['label'] == "negative"]["sent"]:
    if len(re.sub('[\w\s]','',i)) >3:
        spec+=1
    count+=1
print(spec/count)

0.3834226149596325


After some experiment, we found 38% of bad example sentences have more than 3 punctuations. In contrast, only 5.8% of good example sentences have more than 3 punctuations. Hence, it seems that it is a nice feature to distinguish good and bad example sentences, so we add this feature.

In [6]:
train["more than three punc"] = [1 if len(re.sub('[\w\s]','',i)) >3 else 0 for i in train["sent"]]
test["more than three punc"] = [1 if len(re.sub('[\w\s]','',i))>3 else 0 for i in test["sent"]]

In [7]:
train.head()

,sent,label,more than three punc
0,My children threw a birthday party for me.,positive,0
1,Marketing on social networking sites is a fast...,positive,0
2,"You pillock, look what you've done!",positive,0
3,He scored three touchdowns early in the game.,positive,0
4,His abrupt manner makes me uncomfortable.,positive,0


<font color="red">**[ TODO ]**</font> Please observe the data and extract at least three features and add them to the dataframe.

In [102]:
def exceed_limit_uppercase(i):  #以大寫數量區分
    limit = 3
    if(len(re.sub(r'[^A-Z]', '', i)) > limit):
        return True 
    
    return False

In [103]:
train["exceed_limit_uppercase"] = [1 if exceed_limit_uppercase(i) else 0 for i in train["sent"]]
test["exceed_limit_uppercase"] = [1 if exceed_limit_uppercase(i) else 0 for i in test["sent"]]

In [137]:
# Find average length
len_sum = 0
neg = train[train['label'] == "negative"]["sent"]

for i in neg:
    len_sum += len(i.split())
    #print(len(i.split()))
neg_avg_len = len_sum / neg.count()
print(neg_avg_len)


len_sum = 0
pos = train[train['label'] == "positive"]["sent"]

for i in pos:
    len_sum += len(i.split())
    #print(len(i.split()))
pos_avg_len = len_sum / pos.count()
print(pos_avg_len)

def exceed_limit_len_range(i):  #以字串長度區分
    limit = 12
    if (len(i) < limit):
        return True
    
    return False   

15.81438578819651
11.071770207790827


In [138]:
train["exceed_limit_len_range"] = [1 if exceed_limit_len_range(i) else 0 for i in train["sent"]]
test["exceed_limit_len_range"] = [1 if exceed_limit_len_range(i) else 0 for i in test["sent"]]

In [126]:
def exceed_limit_numbers(i):  #已出現數字數量區分
    limit = 3
    if(len(re.sub(r'[^0-9]', '', i)) > limit):
        return True 

    return False

In [127]:
train["exceed_limit_numbers"] = [1 if exceed_limit_numbers(i) else 0 for i in train["sent"]]
test["exceed_limit_numbers"] = [1 if exceed_limit_numbers(i) else 0 for i in test["sent"]]

## Train
Now, it's time to evaluate whether the features just selected is useful to classify. We use [Bernoulli Naive Bayes model](https://scikit-learn.org/stable/modules/naive_bayes.html#bernoulli-naive-bayes) to train training data. 

In [139]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()

<font color="red">**[ TODO ]**</font> Please adjust the `selected_features` list and train the model.

***Don't*** use other model in this assignment.

In [140]:
selected_features = ["more than three punc", "exceed_limit_len_range", "exceed_limit_uppercase", "exceed_limit_numbers"]

In [141]:
y_pred = bnb.fit(train[selected_features], train['label']).predict(test[selected_features])

## Test
Test the model and get the accuracy of the prediction on testing data. 

There're four baselines for this task:
```
1. simple baseline: 0.65
2. medium baseline: 0.72
3. strong baseline: 0.8
4. boss baseline: 0.85
```
The more the baseline you pass, the more higher the grade you can get.

*hint: If the result isn't ideal, you can print the wrong prediction data and re-observe but only focus on wrong data to extract other features.

In [142]:
print((test['label'] == y_pred).sum()/len(test))

0.7661861074705112


## TA's Notes

If you complete the Assignment, please use [this link](https://docs.google.com/spreadsheets/d/1QGeYl5dsD9sFO9SYg4DIKk-xr-yGjRDOOLKZqCLDv2E/edit#gid=1031097651) to reserve demo time.  
The score is only given after TAs review your implementation, so <u>**make sure you make a appointment with a TA before you miss the deadline**</u> .  <br>After demo, please upload your assignment to eeclass. You just need to hand in this ipynb file and rename it as XXXXXXXXX(Your student ID).ipynb.
<br>Note that **late submission will not be allowed**.